In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.log.util import sorting
from scipy.stats import wasserstein_distance
from sklearn import preprocessing

from log_distance_measures.config import EventLogIDs, AbsoluteTimestampType, discretize_to_hour
from log_distance_measures.control_flow_log_distance import control_flow_log_distance
from log_distance_measures.n_gram_distribution import n_gram_distribution_distance
from log_distance_measures.absolute_event_distribution import absolute_event_distribution_distance
from log_distance_measures.case_arrival_distribution import case_arrival_distribution_distance
from log_distance_measures.circadian_event_distribution import circadian_event_distribution_distance
from log_distance_measures.relative_event_distribution import relative_event_distribution_distance
from log_distance_measures.work_in_progress import work_in_progress_distance
from log_distance_measures.cycle_time_distribution import cycle_time_distribution_distance

import warnings
warnings.filterwarnings("ignore")

In [2]:
import re
def extract_first_float(cell):
    if isinstance(cell, str):
        # Use regular expression to extract the first float and the value in brackets
        match = re.match(r'(\d+\.\d+)(?: \((\d+\.\d+)\))?', cell)
        if match:
            return float(match.group(1)), (match.group(2)) if match.group(2) else ''
        else:
            return float('inf'), ''
    else:
        return cell, ''

In [3]:
def highlight_min_max(s):
    """
    Highlight the minimum value in green and the maximum value in red for each column.
    """
    is_min = s == s.min()
    is_max = s == s.max()
    min_max_style = ['background-color: green' if v else '' for v in is_min]
    for i, v in enumerate(is_max):
        if v:
            min_max_style[i] = 'background-color: red'
    return min_max_style

In [4]:
def align_column_names(df):
    if 'case:concept:name' in df.columns:
        df = df.rename(columns={'case:concept:name': 'case_id'})
    elif 'caseid' in df.columns:
        df = df.rename(columns={'caseid': 'case_id'})
    if 'Activity' in df.columns:
        df = df.rename(columns={'Activity': 'activity'})
    elif 'activity_name' in df.columns:
        df = df.rename(columns={'activity_name': 'activity'})
    elif 'task' in df.columns:
        df = df.rename(columns={'task': 'activity'})
    elif 'concept:name' in df.columns:
        df = df.rename(columns={'concept:name': 'activity'})
    if 'Resource' in df.columns:
        df = df.rename(columns={'Resource': 'resource'})
    elif 'user' in df.columns:
        df = df.rename(columns={'user': 'resource'})
    elif 'agent' in df.columns:
        if 'resource' in df.columns:
            df = df.drop(['resource'], axis=1)
        df = df.rename(columns={'agent': 'resource'})
    elif 'org:resource' in df.columns:
        df = df.rename(columns={'org:resource': 'resource'})
    if 'start_timestamp' in df.columns:
        df = df.rename(columns={'start_timestamp': 'start_time'})
    if 'end_timestamp' in df.columns:
        df = df.rename(columns={'end_timestamp': 'end_time'})
    # for SIMOD simulated logs
    if 'start_time' in df.columns:
        df = df.rename(columns={'start_time': 'start_time'})
    if 'end_time' in df.columns:
        df = df.rename(columns={'end_time': 'end_time'})
    if 'start:timestamp' in df.columns:
        df = df.rename(columns={'start:timestamp': 'start_time'})
    if 'time:timestamp' in df.columns:
        df = df.rename(columns={'time:timestamp': 'end_time'})
    return df

In [10]:
def main_(log_paths, name_experiments):
    def perform_evauluation(all_metrics, PATH_SIMULATED_LOG, test_log):
        for i in range(10):
            # print(f"Evaluate simulation {i}")
            path_simulated_file = PATH_SIMULATED_LOG + '/simulated_log_' + str(i) + '.csv'
            # read simulated log and align column names
            simulated_log = pd.read_csv(path_simulated_file)
            simulated_log = align_column_names(simulated_log)
            # print(simulated_log)
            # print("########")
            # print(simulated_log[event_log_ids.activity].unique())
            simulated_log[event_log_ids.start_time] = pd.to_datetime(simulated_log[event_log_ids.start_time], utc=True, format='mixed')
            simulated_log[event_log_ids.end_time] = pd.to_datetime(simulated_log[event_log_ids.end_time], utc=True, format='mixed')

            # Call passing the event logs, and its column ID mappings
            ngd = n_gram_distribution_distance(test_log, event_log_ids, simulated_log, event_log_ids, n=3)
            all_metrics['NGD'].append(ngd)

            # Call passing the event logs, its column ID mappings, timestamp type, and discretize function
            aedd = absolute_event_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
                discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
            )
            all_metrics['AEDD'].append(aedd)

            cadd = case_arrival_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_event=discretize_to_hour  # Function to discretize each timestamp (default by hour)
            )
            all_metrics['CADD'].append(cadd)

            cedd = circadian_event_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
            )
            all_metrics['CEDD'].append(cedd)

            redd = relative_event_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
                discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
            )
            all_metrics['REDD'].append(redd)


            ctdd = cycle_time_distribution_distance(
                test_log, event_log_ids,  # First event log and its column id mappings
                simulated_log, event_log_ids,  # Second event log and its column id mappings
                bin_size=pd.Timedelta(hours=1)  # Bins of 1 hour
            )
            all_metrics['CTDD'].append(ctdd)

        return all_metrics
    
    number_evaluations = len(log_paths)

    # Set event log column ID mapping
    event_log_ids = EventLogIDs(  # These values are stored in DEFAULT_CSV_IDS
        case="case_id",
        activity="activity",
        start_time="start_time",
        end_time="end_time",
        resource='resource'
    )

    index_names = name_experiments
    results_df = pd.DataFrame(index=index_names)
    mean_results = pd.DataFrame(index=index_names)

    for experiment in range(number_evaluations):
        # Read and transform time attributes
        test_log = pd.read_csv(log_paths[experiment][0])
        test_log = align_column_names(test_log)
        test_log[event_log_ids.start_time] = pd.to_datetime(test_log[event_log_ids.start_time], utc=True, format='mixed')
        test_log[event_log_ids.end_time] = pd.to_datetime(test_log[event_log_ids.end_time], utc=True, format='mixed')

        PATH_SIMULATED_LOG = log_paths[experiment][1]

        all_metrics = {
            'NGD': [],
            'AEDD': [],
            'CADD': [],
            'CEDD': [],
            'REDD': [],
            'CTDD': [],
        }

        all_metrics = perform_evauluation(all_metrics, PATH_SIMULATED_LOG, test_log)

        mean_results.loc[index_names[experiment], 'N-Gram Distribution Distance'] = round(np.mean(all_metrics['NGD']), 3)
        mean_results.loc[index_names[experiment], 'Absolute Event Distribution Distance'] = round(np.mean(all_metrics['AEDD']), 3)
        mean_results.loc[index_names[experiment], 'Case Arrival Distribution Distance'] = round(np.mean(all_metrics['CADD']), 3)
        mean_results.loc[index_names[experiment], 'Circadian Event Distribution Distance'] = round(np.mean(all_metrics['CEDD']), 3)
        mean_results.loc[index_names[experiment], 'Relative Event Distribution Distance'] = round(np.mean(all_metrics['REDD']), 3)
        mean_results.loc[index_names[experiment], 'Cycle Time Distribution Distance'] = round(np.mean(all_metrics['CTDD']), 3)


    

        results_df.loc[index_names[experiment], 'N-Gram Distribution Distance'] = f"{round(np.mean(all_metrics['NGD']), 3)} ({round(np.std(all_metrics['NGD']), 3)})"
        results_df.loc[index_names[experiment], 'Absolute Event Distribution Distance'] = f"{round(np.mean(all_metrics['AEDD']), 3)} ({round(np.std(all_metrics['AEDD']), 3)})"
        results_df.loc[index_names[experiment], 'Case Arrival Distribution Distance'] = f"{round(np.mean(all_metrics['CADD']), 3)} ({round(np.std(all_metrics['CADD']), 3)})"
        results_df.loc[index_names[experiment], 'Circadian Event Distribution Distance'] = f"{round(np.mean(all_metrics['CEDD']), 3)} ({round(np.std(all_metrics['CEDD']), 3)})"
        results_df.loc[index_names[experiment], 'Relative Event Distribution Distance'] = f"{round(np.mean(all_metrics['REDD']), 3)} ({round(np.std(all_metrics['REDD']), 3)})"
        results_df.loc[index_names[experiment], 'Cycle Time Distribution Distance'] = f"{round(np.mean(all_metrics['CTDD']), 3)} ({round(np.std(all_metrics['CTDD']), 3)})"

    return mean_results, results_df


# Loan Application

In [23]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/Loan_Application_simod/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/Loan_Application_simod/best_result/evaluation'

PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/Loan_Application_extraneous/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/Loan_Application_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/Loan_Application/tst_LoanApp.csv'
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/Loan_Application'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/Loan_Application/tst_SynLoan.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/Loan_Application'

PATH_TEST_LOG_MAS = '../simulated_data/LoanApp.csv/no_delays/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/LoanApp.csv/no_delays'

PATH_TEST_LOG_MAS_extr = '../simulated_data/LoanApp.csv/delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/LoanApp.csv/delays_decentral/'

PATH_TEST_LOG_MAS_central = '../simulated_data/LoanApp.csv/no_delays_central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_central = '../simulated_data/LoanApp.csv/no_delays_central/'


log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], [PATH_TEST_LOG_SIMOD_extr, PATH_SIMULATED_LOG_SIMOD_extr], 
             [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM],
                [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS], [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr],
                [PATH_TEST_LOG_MAS_central, PATH_SIMULATED_LOG_MAS_central]]

name_experiments = ['SIMOD', 'SIMOD_extr', 'DGEN', 'DSIM', 'MAS', 'MAS_extr', 'MAS_central']

In [24]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.210000,12.344000,15.797000,0.418000,12.865000,33.997000
SIMOD_extr,0.150000,13.551000,17.393000,0.402000,9.228000,20.425000
DGEN,0.219000,212.278000,206.700000,13.403000,5.262000,9.385000
DSIM,0.557000,428.562000,326.035000,0.376000,109.816000,467.776000
MAS,0.077000,5.231000,6.700000,0.206000,1.314000,1.584000
MAS_extr,0.074000,2.936000,0.004000,0.199000,1.662000,1.749000
MAS_central,0.075000,2.761000,0.000000,0.219000,1.358000,1.503000


## P2P

In [35]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/P2P_simod/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/P2P_simod/best_result/evaluation'

PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/P2P_extraneous/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/P2P_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/P2P/tst_PurchasingExample.csv'
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/P2P'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/PurchasingExample/tst_PurchasingExample.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/PurchasingExample'

PATH_TEST_LOG_MAS = '../simulated_data/P2P/decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/P2P/decentral'

PATH_TEST_LOG_MAS_extr = '../simulated_data/P2P/delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/P2P/delays_decentral/'

PATH_TEST_LOG_MAS_extr_central = '../simulated_data/P2P/delays_central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr_central = '../simulated_data/P2P/delays_central/'


log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], [PATH_TEST_LOG_SIMOD_extr, PATH_SIMULATED_LOG_SIMOD_extr], 
                [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM], 
             [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS], [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr], 
             [PATH_TEST_LOG_MAS_extr_central, PATH_SIMULATED_LOG_MAS_extr_central], ]

name_experiments = ['SIMOD', 'SIMOD_extr', 'DGEN', 'DSIM', 'MAS', 'MAS_extr', 'MAS_extr_central']

In [36]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.415000,1044.251000,563.892000,2.213000,840.194000,677.053000
SIMOD_extr,0.415000,867.561000,556.312000,1.674000,682.133000,571.583000
DGEN,0.200000,1481.460000,1007.446000,2.558000,828.093000,670.053000
DSIM,0.222000,1310.037000,927.584000,1.157000,722.334000,566.637000
MAS,0.237000,1272.434000,857.309000,4.799000,766.139000,546.851000
MAS_extr,0.252000,1122.419000,811.832000,0.976000,668.205000,529.084000
MAS_extr_central,0.249000,1098.160000,779.446000,1.065000,674.828000,528.991000


## ACR = ConsultaDataMining

In [33]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/ConsultaDataMining_simod/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/ConsultaDataMining_simod/best_result/evaluation'

PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/ConsultaDataMining_extraneous/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/ConsultaDataMining_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/ConsultaDataMining/tst_ConsultaDataMining201618.csv'
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/ConsultaDataMining'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/ConsultaDataMining201618/tst_ConsultaDataMining201618.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/ConsultaDataMining201618'

PATH_TEST_LOG_MAS = '../simulated_data/ConsultaDataMining/decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/ConsultaDataMining/decentral'

PATH_TEST_LOG_MAS_extr = '../simulated_data/ConsultaDataMining/delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/ConsultaDataMining/delays_decentral'

PATH_TEST_LOG_MAS_central = '../simulated_data/ConsultaDataMining/central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_central = '../simulated_data/ConsultaDataMining/central'


log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], [PATH_TEST_LOG_SIMOD_extr, PATH_SIMULATED_LOG_SIMOD_extr], 
    [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM], 
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS], [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr], [PATH_TEST_LOG_MAS_central, PATH_SIMULATED_LOG_MAS_central]]

name_experiments = ['SIMOD', 'SIMOD_extr', 'DGEN', 'DSIM', 'MAS', 'MAS_extr', 'MAS_central']

In [34]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.239000,287.279000,252.235000,2.602000,32.461000,93.512000
SIMOD_extr,0.501000,107.697000,164.108000,3.156000,94.391000,277.454000
DGEN,0.314000,559.675000,527.891000,17.840000,30.875000,95.113000
DSIM,0.260000,273.468000,262.327000,4.644000,15.621000,48.246000
MAS,0.524000,305.226000,282.523000,6.953000,26.046000,75.649000
MAS_extr,0.394000,237.275000,361.273000,5.702000,168.656000,407.110000
MAS_central,0.367000,281.576000,254.927000,7.212000,27.328000,77.260000


## Production

In [39]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/Production/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/Production/best_result/evaluation'

PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/Production_extraneous/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/Production_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/Productions/tst_Productions.csv'
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/Productions'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/Production/tst_Production.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/Production'

PATH_TEST_LOG_MAS = '../simulated_data/Production/no_delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/Production/no_delays_decentral'

PATH_TEST_LOG_MAS_extr = '../simulated_data/Production/decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/Production/decentral'

PATH_TEST_LOG_MAS_extr_central = '../simulated_data/Production/central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr_central = '../simulated_data/Production/central'

PATH_TEST_LOG_MAS_central = '../simulated_data/Production/no_delays_central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_central = '../simulated_data/Production/no_delays_central'

log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], [PATH_TEST_LOG_SIMOD_extr, PATH_SIMULATED_LOG_SIMOD_extr], [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM], 
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS], [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr], [PATH_TEST_LOG_MAS_central, PATH_SIMULATED_LOG_MAS_central], 
    [PATH_TEST_LOG_MAS_extr_central,PATH_SIMULATED_LOG_MAS_extr_central] ]
name_experiments = ['SIMOD', 'SIMOD_extr', 'DGEN', 'DSIM', 'MAS', 'MAS_extr', 'MAS_central', 'MAS_extr_central']

In [40]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.928000,146.381000,166.895000,2.820000,83.880000,89.150000
SIMOD_extr,0.926000,309.040000,177.200000,3.236000,78.786000,212.898000
DGEN,0.528000,224.455000,198.333000,9.300000,70.113000,90.820000
DSIM,0.868000,154.319000,165.609000,2.669000,33.308000,43.267000
MAS,0.774000,70.358000,79.034000,5.970000,22.491000,40.671000
MAS_extr,0.745000,148.332000,78.292000,3.131000,193.250000,237.108000
MAS_central,0.615000,61.803000,80.616000,5.688000,17.773000,23.450000
MAS_extr_central,0.628000,54.918000,92.203000,5.778000,13.309000,33.795000


## BPIC 2012 W

In [110]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/BPIC_2012W_simod/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/BPIC_2012W_simod/best_result/evaluation'

PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/BPIC_2012W_extraneous/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/BPIC_2012W_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/BPIC_2012W/tst_BPI_Challenge_2012_W_Two_TS.csv'
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/BPIC_2012W'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/BPI_Challenge_2012_W_Two_TS/tst_BPI_Challenge_2012_W_Two_TS.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/BPI_Challenge_2012_W_Two_TS'

PATH_TEST_LOG_MAS = '../simulated_data/BPIC_2012_W/no_delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/BPIC_2012_W/no_delays_decentral'

PATH_TEST_LOG_MAS_extr = '../simulated_data/BPIC_2012_W/decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/BPIC_2012_W/decentral'

PATH_TEST_LOG_MAS_extr_central = '../simulated_data/BPIC_2012_W/central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr_central = '../simulated_data/BPIC_2012_W/central'

log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], [PATH_TEST_LOG_SIMOD_extr, PATH_SIMULATED_LOG_SIMOD_extr], 
    [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM], [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr], 
    [PATH_TEST_LOG_MAS_extr_central, PATH_SIMULATED_LOG_MAS_extr_central]]

name_experiments = ['SIMOD', 'SIMOD_extr', 'DGEN', 'DSIM', 'MAS', 'MAS_extr', 'MAS_extr_central']

In [111]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.633000,120.262000,17.013000,1.876000,140.684000,194.671000
SIMOD_extr,0.720000,71.977000,18.128000,1.716000,95.720000,155.467000
DGEN,0.435000,306.280000,199.315000,4.534000,116.188000,176.794000
DSIM,0.655000,78.625000,32.128000,2.887000,119.126000,173.491000
MAS,0.322000,82.375000,50.508000,2.163000,139.595000,194.403000
MAS_extr,0.213000,92.010000,44.503000,1.915000,52.054000,96.909000
MAS_extr_central,0.151000,81.523000,35.903000,1.893000,49.609000,92.993000


## CVS Pharmacy

In [23]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/cvs_pharmacy_simod/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/cvs_pharmacy_simod/best_result/evaluation'

PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/cvs_pharmacy_extraneous/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/cvs_pharmacy_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/CVS/tst_cvs_pharmacy.csv'
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/CVS'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/CVS/tst_cvs_pharmacy.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/CVS'

PATH_TEST_LOG_MAS = '../simulated_data/cvs_pharmacy/delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/cvs_pharmacy/delays_decentral'

PATH_TEST_LOG_MAS_extr = '../simulated_data/cvs_pharmacy/decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/cvs_pharmacy/decentral'

PATH_TEST_LOG_MAS_extr_central = '../simulated_data/cvs_pharmacy/central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr_central = '../simulated_data/cvs_pharmacy/central'


log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], [PATH_TEST_LOG_SIMOD_extr, PATH_SIMULATED_LOG_SIMOD_extr], 
    [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM], [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr], [PATH_TEST_LOG_MAS_extr_central, PATH_SIMULATED_LOG_MAS_extr_central] ]
name_experiments = ['SIMOD', 'SIMOD_extr', 'DGEN', 'DSIM', 'MAS', 'MAS_extr', 'MAS_extr_central']

In [24]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.440000,52.947000,7.709000,0.444000,39.432000,54.594000
SIMOD_extr,0.465000,45.774000,6.432000,1.057000,37.557000,90.999000
DGEN,0.219000,310.394000,133.461000,11.699000,176.652000,294.214000
DSIM,0.201000,36.237000,20.367000,8.982000,19.743000,52.430000
MAS,0.086000,373.835000,205.725000,0.395000,168.234000,273.227000
MAS_extr,0.119000,91.816000,2.635000,7.475000,87.738000,109.702000
MAS_extr_central,0.118000,94.986000,5.122000,7.460000,88.657000,111.070000


## BPIC 2017

In [45]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/BPIC_2017_W_simod/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/BPIC_2017_W_simod/best_result/evaluation'

PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/BPIC_2017_W_extraneous/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/BPIC_2017_W_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/BPIC_2017W/test_log.csv' 
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/BPIC_2017W'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/BPI_Challenge_2017_W_Two_TS/tst_BPI_Challenge_2017_W_Two_TS.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/BPI_Challenge_2017_W_Two_TS'

PATH_TEST_LOG_MAS = '../simulated_data/BPIC_2017_W/no_delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/BPIC_2017_W/no_delays_decentral'

PATH_TEST_LOG_MAS_extr = '../simulated_data/BPIC_2017_W/decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/BPIC_2017_W/decentral'

PATH_TEST_LOG_MAS_extr_central = '../simulated_data/BPIC_2017_W/central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr_central = '../simulated_data/BPIC_2017_W/central'



log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], [PATH_TEST_LOG_SIMOD_extr, PATH_SIMULATED_LOG_SIMOD_extr], [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], 
    [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM], [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr], [PATH_TEST_LOG_MAS_extr_central,PATH_SIMULATED_LOG_MAS_extr_central],
    ]
name_experiments = ['SIMOD', 'SIMOD_extr', 'DGEN', 'DSIM', 'MAS', 'MAS_extr', 'MAS_extr_central']

In [46]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.591000,385.129000,194.537000,2.258000,208.852000,274.698000
SIMOD_extr,0.591000,300.280000,182.133000,3.342000,136.630000,148.403000
DGEN,0.671000,4557.193000,4594.980000,3.396000,118.848000,172.940000
DSIM,0.536000,54.613000,53.989000,3.346000,33.106000,30.266000
MAS,0.194000,430.907000,240.228000,1.525000,209.105000,275.302000
MAS_extr,0.352000,380.512000,203.818000,1.677000,400.113000,34.997000
MAS_extr_central,0.193000,221.493000,193.537000,1.787000,50.010000,54.819000


## Confidential 1000

In [41]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/Confidential_1000_simod/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/Confidential_1000_simod/best_result/evaluation'

PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/Confidential_1000_extraneous/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/Confidential_1000_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/Confidential_1000/tst_confidential_1000.csv'
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/Confidential_1000'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/confidential_1000/tst_confidential_1000.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/confidential_1000'

PATH_TEST_LOG_MAS = '../simulated_data/Confidential_1000/decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/Confidential_1000/decentral'

PATH_TEST_LOG_MAS_extr = '../simulated_data/Confidential_1000/delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/Confidential_1000/delays_decentral'

PATH_TEST_LOG_MAS_central = '../simulated_data/Confidential_1000/central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_central = '../simulated_data/Confidential_1000/central'



log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], [PATH_TEST_LOG_SIMOD_extr, PATH_SIMULATED_LOG_SIMOD_extr], 
    [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM], [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr], [PATH_TEST_LOG_MAS_central,PATH_SIMULATED_LOG_MAS_central],
    ]
name_experiments = ['SIMOD', 'SIMOD_extr', 'DGEN', 'DSIM', 'MAS', 'MAS_extr', 'MAS_central']

In [42]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.247000,344.486000,119.326000,3.010000,468.818000,804.074000
SIMOD_extr,0.243000,351.939000,119.448000,2.725000,476.036000,813.441000
DGEN,0.581000,462.847000,452.122000,18.934000,8.114000,13.929000
DSIM,0.203000,246.410000,239.415000,2.280000,5.342000,7.297000
MAS,0.266000,166.738000,171.040000,1.702000,6.939000,9.494000
MAS_extr,0.188000,123.083000,207.147000,1.937000,85.425000,139.547000
MAS_central,0.255000,135.703000,145.928000,1.660000,13.221000,23.155000


## Confidential 2000

In [57]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/Confidential_2000_simod/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/Confidential_2000_simod/best_result/evaluation'

PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/Confidential_2000_extraneous/best_result/evaluation/test_log.csv'
PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/Confidential_2000_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/Confidential_2000/tst_confidential_2000.csv'
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/Confidential_2000'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/confidential_2000/tst_confidential_2000.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/confidential_2000'

PATH_TEST_LOG_MAS = '../simulated_data/Confidential_2000/no_delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/Confidential_2000/no_delays_decentral'

PATH_TEST_LOG_MAS_extr = '../simulated_data/Confidential_2000/delays/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/Confidential_2000/delays'

PATH_TEST_LOG_MAS_central = '../simulated_data/Confidential_2000/no_delays_central/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_central = '../simulated_data/Confidential_2000/no_delays_central'



log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], [PATH_TEST_LOG_SIMOD_extr, PATH_SIMULATED_LOG_SIMOD_extr], 
    [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM], [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr], [PATH_TEST_LOG_MAS_central,PATH_SIMULATED_LOG_MAS_central],
    ]
name_experiments = ['SIMOD', 'SIMOD_extr', 'DGEN', 'DSIM', 'MAS', 'MAS_extr', 'MAS_central']

In [58]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.246000,820.454000,148.235000,2.968000,952.378000,1614.918000
SIMOD_extr,0.244000,816.741000,184.304000,3.116000,982.095000,1662.365000
DGEN,0.161000,857.683000,876.657000,18.092000,4.588000,8.120000
DSIM,0.186000,591.136000,613.292000,2.845000,1.701000,2.262000
MAS,0.261000,263.253000,291.694000,1.276000,9.192000,17.311000
MAS_extr,0.192000,130.789000,176.112000,1.657000,30.924000,48.224000
MAS_central,0.253000,236.912000,266.964000,1.490000,9.302000,17.729000


# SynLoan

In [39]:
PATH_TEST_LOG_SIMOD = '../simulated_data/simod/SynLoan/test_log.csv'
PATH_SIMULATED_LOG_SIMOD = '../simulated_data/simod/SynLoan'

# PATH_TEST_LOG_SIMOD_extr = '../simulated_data/simod/Confidential_2000_extraneous/best_result/evaluation/test_log.csv'
# PATH_SIMULATED_LOG_SIMOD_extr = '../simulated_data/simod/Confidential_2000_extraneous/best_result/evaluation'

PATH_TEST_LOG_DGEN = '../simulated_data/deep_generator/SynLoan/test_log.csv'
PATH_SIMULATED_LOG_DGEN = '../simulated_data/deep_generator/SynLoan'

PATH_TEST_LOG_DSIM = '../simulated_data/deep_simulator/SynLoan/test_log.csv'
PATH_SIMULATED_LOG_DSIM = '../simulated_data/deep_simulator/SynLoan'

PATH_TEST_LOG_MAS = '../simulated_data/SynLoan/no_delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS = '../simulated_data/SynLoan/no_delays_decentral'

PATH_TEST_LOG_MAS_extr = '../simulated_data/SynLoan/delays_decentral/test_preprocessed.csv'
PATH_SIMULATED_LOG_MAS_extr = '../simulated_data/SynLoan/delays_decentral'

# PATH_TEST_LOG_MAS_central = '../simulated_data/SynLoan/no_delays_central/test_preprocessed.csv'
# PATH_SIMULATED_LOG_MAS_central = '../simulated_data/SynLoan/no_delays_central'



log_paths = [[PATH_TEST_LOG_SIMOD, PATH_SIMULATED_LOG_SIMOD], 
    [PATH_TEST_LOG_DGEN, PATH_SIMULATED_LOG_DGEN], [PATH_TEST_LOG_DSIM, PATH_SIMULATED_LOG_DSIM], 
    [PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS],
    [PATH_TEST_LOG_MAS_extr, PATH_SIMULATED_LOG_MAS_extr], 
    # [PATH_TEST_LOG_MAS_central,PATH_SIMULATED_LOG_MAS_central],
    ]
name_experiments = ['SIMOD', 'DGEN', 'DSIM','MAS', 'MAS_extr']#, 'MAS_central']

In [40]:
mean_results, results_df = main_(log_paths, name_experiments)
styled_df = mean_results.style.apply(highlight_min_max)
styled_df

,N-Gram Distribution Distance,Absolute Event Distribution Distance,Case Arrival Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
SIMOD,0.700000,869.651000,344.959000,0.877000,513.486000,637.680000
DGEN,0.396000,915.207000,554.565000,3.716000,349.326000,402.954000
DSIM,0.557000,428.562000,326.035000,0.376000,109.816000,467.776000
MAS,0.104000,448.929000,6.601000,0.483000,453.119000,586.861000
MAS_extr,0.102000,448.215000,3.073000,0.509000,451.922000,584.421000
